# TAXII Feed Data Filtering
This notebook demonstrates useful ways of filtering out specific malware indicator data from the ReversingLabs TAXII Feed collections.
Currently, our TAXII Feed as a STIX object feed for our Ransomware Indicators Feed and our Flexible Intel Feeds service.

### Used Spectra Intelligence classes
- **TAXIIFeed**

### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)

In [1]:
import json
from datetime import datetime, timedelta
from ReversingLabs.SDK.ticloud import TAXIIFeed


CREDENTIALS = json.load(open("credentials.json"))
USERNAME = CREDENTIALS.get("ticloud").get("username")
PASSWORD = CREDENTIALS.get("ticloud").get("password")
USER_AGENT = json.load(open('../user_agent.json'))["user_agent"]


taxii = TAXIIFeed(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

## Latest CobaltStrike indicators with high confidence from Ransomware Indicators Feed data
We will now fetch Ransomware Feed data from the last 30 days of time and filter out CobaltStrike indicators with confidence level of 80 or more.

In [ ]:
discovery_response = taxii.discovery_info()
print(discovery_response.text)

The response output of the `discovery_info` method will list the API roots to which your username has access.
In this example, we will work with the `ransomware-api-root` API root which holds full Ransomware Indicators Feed data. You can also work with the lite version of that data found in the `ransomware-lite` API root.

In [ ]:
api_root_info = taxii.api_root_info("ransomware-api-root")
print(api_root_info.text)

collection_info = taxii.collections_info("ransomware-api-root")
print(collection_info.text)

Now that we see that our `ransomware-api-root` holds a collection with the id `f0997a32-b823-562d-9856-c754ac5e1159`, we can use that collection for fetching STIX data.
We will set `max_results` to 500 to limit our data output because 30 days of data can be a lot.
You can always adjust the starting date and the maximum number of returned results.

The printed-out results will give us indicators from our Ransomware Indicators Feed for CobaltStrike activity with a confidence rating of 80 and higher.

In [ ]:
time_now = datetime.now()
requested_time = (time_now - timedelta(days=30)).strftime("%Y-%m-%dT%H:%M:%SZ")

response = taxii.get_objects_aggregated(
    api_root="ransomware-api-root",
    collection_id="f0997a32-b823-562d-9856-c754ac5e1159",
    added_after=requested_time,
    stix_types="indicator",
    labels=["CobaltStrike"],
    confidence=">=80",
    max_results=500
)

print(response)

## Latest CobaltStrike objects with high confidence from Flexible Intel Feeds data
We will now apply the same filtering criteria to get CobaltStrike-related data generated by our Flexible Intel Feeds service. The Flexible Intel Feeds service also creates other types of STIX objects, besides indicators.

**NOTE**: The Flexible Intel Feeds is an on-demand indicator processing service and it creates a private collection for each specific user. You will have to replace the `collection_id` with the id of your own FIF collection.

In [ ]:
time_now = datetime.now()
requested_time = (time_now - timedelta(days=30)).strftime("%Y-%m-%dT%H:%M:%SZ")

response = taxii.get_objects_aggregated(
    api_root="flexible-intel-feeds",
    collection_id="your-collection-id-here",
    added_after=requested_time,
    stix_types=["indicator", "url", "file", "domain-name", "ipv4-addr"],
    labels=["CobaltStrike"],
    confidence=">=80",
    max_results=500
)

print(response)

## Latest Phishing domains with high confidence from Flexible Intel Feeds data


In [ ]:
time_now = datetime.now()
requested_time = (time_now - timedelta(days=30)).strftime("%Y-%m-%dT%H:%M:%SZ")

response = taxii.get_objects_aggregated(
    api_root="flexible-intel-feeds",
    collection_id="your-collection-id-here",
    added_after=requested_time,
    stix_types="domain-name",
    labels=["Phishing"],
    confidence=">=80",
    max_results=500
)

print(response)

## Latest Trojan malware data with high confidence from Ransomware Indicators Feed data
Here we will filter out latest Trojan data labeled with PE/Exe (Windows executables) from our Ransomware Indicators Feed.

In [ ]:
time_now = datetime.now()
requested_time = (time_now - timedelta(days=30)).strftime("%Y-%m-%dT%H:%M:%SZ")

response = taxii.get_objects_aggregated(
    api_root="ransomware-api-root",
    collection_id="f0997a32-b823-562d-9856-c754ac5e1159",
    added_after=requested_time,
    stix_types="indicator",
    labels=["Trojan", "PE/Exe"],
    confidence=">=80",
    max_results=500
)

print(json.dumps(response))

### Example output
The returned data is a Python list of dictionary objects. Below is one example object dumped as a JSON string.
<details>
<summary>Expand to see the JSON example</summary>

JSON object example
```json
[
    {
        "type": "indicator",
        "spec_version": "2.1",
        "id": "indicator--9d097aab-bf8d-5915-9864-0e0f235b81ad",
        "created": "2025-05-20T16:12:13Z",
        "modified": "2025-05-20T16:12:13Z",
        "valid_from": "2025-05-20T16:12:13Z",
        "valid_until": "2025-06-19T16:12:13Z",
        "created_by_ref": "identity--ef2a68d1-a8d2-5cb4-973d-1b9e0858c41e",
        "name": "Malware File",
        "description": "Malware file activity was observed",
        "confidence": 100,
        "indicator_types": [
            "malicious-activity"
        ],
        "pattern_type": "stix",
        "pattern": "[file:hashes.'SHA-1' = 'b2ce7b5d50cad456498bc149f96fba2cd7d33033' OR file:hashes.'MD5' = 'a56f71b1a3a5b75f7a85de1cb43bd5e0' OR file:hashes.'SHA-256' = '82617a84d35721a149581353decda3ba78fb65b4ddeb9dc3ebdddcd855e6652f']",
        "labels": [
            "ReversingLabs",
            "1950208",
            "PE/Exe",
            "DBatLoader_a56f71b1a3a5b75f7a85de1cb43bd5e0.exe",
            "Remcos",
            "Trojan",
            "Early"
        ],
        "external_references": [
            {
                "source_name": "mitre",
                "external_id": "T1003",
                "description": "OS Credential Dumping",
                "url": "https://attack.mitre.org/techniques/T1003/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1007",
                "description": "System Service Discovery",
                "url": "https://attack.mitre.org/techniques/T1007/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1010",
                "description": "Application Window Discovery",
                "url": "https://attack.mitre.org/techniques/T1010/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1012",
                "description": "Query Registry",
                "url": "https://attack.mitre.org/techniques/T1012/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1016",
                "description": "System Network Configuration Discovery",
                "url": "https://attack.mitre.org/techniques/T1016/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1018",
                "description": "Remote System Discovery",
                "url": "https://attack.mitre.org/techniques/T1018/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1027",
                "description": "Obfuscated Files or Information",
                "url": "https://attack.mitre.org/techniques/T1027/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1033",
                "description": "System Owner/User Discovery",
                "url": "https://attack.mitre.org/techniques/T1033/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1036",
                "description": "Masquerading",
                "url": "https://attack.mitre.org/techniques/T1036/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1053",
                "description": "Scheduled Task/Job",
                "url": "https://attack.mitre.org/techniques/T1053/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1055",
                "description": "Process Injection",
                "url": "https://attack.mitre.org/techniques/T1055/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1056",
                "description": "Input Capture",
                "url": "https://attack.mitre.org/techniques/T1056/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1056.004",
                "description": "Credential API Hooking",
                "url": "https://attack.mitre.org/techniques/T1056.004/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1057",
                "description": "Process Discovery",
                "url": "https://attack.mitre.org/techniques/T1057/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1059",
                "description": "Command and Scripting Interpreter",
                "url": "https://attack.mitre.org/techniques/T1059/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1068",
                "description": "Exploitation for Privilege Escalation",
                "url": "https://attack.mitre.org/techniques/T1068/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1071",
                "description": "Application Layer Protocol",
                "url": "https://attack.mitre.org/techniques/T1071/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1078",
                "description": "Valid Accounts",
                "url": "https://attack.mitre.org/techniques/T1078/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1082",
                "description": "System Information Discovery",
                "url": "https://attack.mitre.org/techniques/T1082/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1083",
                "description": "File and Directory Discovery",
                "url": "https://attack.mitre.org/techniques/T1083/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1087",
                "description": "Account Discovery",
                "url": "https://attack.mitre.org/techniques/T1087/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1095",
                "description": "Non-Application Layer Protocol",
                "url": "https://attack.mitre.org/techniques/T1095/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1105",
                "description": "Ingress Tool Transfer",
                "url": "https://attack.mitre.org/techniques/T1105/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1106",
                "description": "Native API",
                "url": "https://attack.mitre.org/techniques/T1106/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1112",
                "description": "Modify Registry",
                "url": "https://attack.mitre.org/techniques/T1112/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1113",
                "description": "Screen Capture",
                "url": "https://attack.mitre.org/techniques/T1113/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1115",
                "description": "Clipboard Data",
                "url": "https://attack.mitre.org/techniques/T1115/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1120",
                "description": "Peripheral Device Discovery",
                "url": "https://attack.mitre.org/techniques/T1120/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1124",
                "description": "System Time Discovery",
                "url": "https://attack.mitre.org/techniques/T1124/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1134",
                "description": "Access Token Manipulation",
                "url": "https://attack.mitre.org/techniques/T1134/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1140",
                "description": "Deobfuscate/Decode Files or Information",
                "url": "https://attack.mitre.org/techniques/T1140/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1218.011",
                "description": "Rundll32",
                "url": "https://attack.mitre.org/techniques/T1218.011/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1219",
                "description": "Remote Access Software",
                "url": "https://attack.mitre.org/techniques/T1219/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1491",
                "description": "Defacement",
                "url": "https://attack.mitre.org/techniques/T1491/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1497",
                "description": "Virtualization/Sandbox Evasion",
                "url": "https://attack.mitre.org/techniques/T1497/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1497.003",
                "description": "Time Based Evasion",
                "url": "https://attack.mitre.org/techniques/T1497.003/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1518.001",
                "description": "Security Software Discovery",
                "url": "https://attack.mitre.org/techniques/T1518.001/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1529",
                "description": "System Shutdown/Reboot",
                "url": "https://attack.mitre.org/techniques/T1529/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1543.003",
                "description": "Windows Service",
                "url": "https://attack.mitre.org/techniques/T1543.003/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1548.002",
                "description": "Bypass User Account Control",
                "url": "https://attack.mitre.org/techniques/T1548.002/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1552.001",
                "description": "Credentials In Files",
                "url": "https://attack.mitre.org/techniques/T1552.001/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1560",
                "description": "Archive Collected Data",
                "url": "https://attack.mitre.org/techniques/T1560/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1562.001",
                "description": "Disable or Modify Tools",
                "url": "https://attack.mitre.org/techniques/T1562.001/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1569.002",
                "description": "Service Execution",
                "url": "https://attack.mitre.org/techniques/T1569.002/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1571",
                "description": "Non-Standard Port",
                "url": "https://attack.mitre.org/techniques/T1571/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1573",
                "description": "Encrypted Channel",
                "url": "https://attack.mitre.org/techniques/T1573/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1574.002",
                "description": "DLL Side-Loading",
                "url": "https://attack.mitre.org/techniques/T1574.002/"
            },
            {
                "source_name": "mitre",
                "external_id": "T1614.001",
                "description": "System Language Discovery",
                "url": "https://attack.mitre.org/techniques/T1614.001/"
            }
        ],
        "kill_chain_phases": [
            {
                "kill_chain_name": "rl-ransomware-kill-chain",
                "phase_name": "early"
            }
        ],
        "revoked": false
    }
```
</details>


## Searching for a specific file in Flexible Intel Feeds using its SHA-1
We can also filter out files by their SHA-1 hash used in the STIX object's `name` field.  
Edit the `name` argument with your desired SHA-1 hash.

In [ ]:
time_now = datetime.now()
requested_time = (time_now - timedelta(days=30)).strftime("%Y-%m-%dT%H:%M:%SZ")

response = taxii.get_objects_aggregated(
    api_root="flexible-intel-feeds",
    collection_id="your-collection-id-here",
    added_after=requested_time,
    stix_types="file",
    confidence=">=80",
    name="example-sha1-hash-here"
)

print(response)

## Searching for a specific object by its ID
Another way to search for a specific object is by using its `id` field.  
For this usecase we will be utilizing the `get_objects` method because we don't need automated paging. This request targets and returns only one specific object matched by its `id` field.

In [ ]:
time_now = datetime.now()
requested_time = (time_now - timedelta(days=30)).strftime("%Y-%m-%dT%H:%M:%SZ")

response = taxii.get_objects(
    api_root="ransomware-api-root",
    collection_id="f0997a32-b823-562d-9856-c754ac5e1159",
    added_after=requested_time,
    match_id="indicator--65dae84d-f2ce-5db3-a4ae-2540c2258a24"
)

print(response.text)